# МОДУЛЬ 7: Многопоточность, асинхронка, параллельность

## 1. Основные концепции

### 1.1. Многозадачность и параллельность
- **Многозадачность**: способность компьютера выполнять несколько задач «одновременно» путём **быстрого переключения** между ними.
- **Параллельность**: фактическое одновременное выполнение нескольких задач, обычно на разных ядрах процессора.

При одноядерных системах создаётся **иллюзия** одновременного выполнения (по времени), тогда как при многоядерных можно действительно исполнять несколько потоков параллельно.

### 1.2. Вытесняющая vs Кооперативная многозадачность
- **Вытесняющая многозадачность**: операционная система самостоятельно решает, когда переключаться между задачами (прерывая текущую).
- **Кооперативная многозадачность**: задачи сами должны уступать управление (пример — `asyncio` в Python).

### 1.3. GIL (Global Interpreter Lock)
- **GIL** — механизм в CPython, позволяющий в каждый момент времени **только одному** потоку исполнять **байт-код Python**.
- Из-за GIL потоки Python **не** выполняются параллельно на уровне байт-кода (хотя могут освобождать GIL при вызове C/IO-операций).

In [1]:
# Код-иллюстрация GIL
# Запускаем два потока с числовыми вычислениями - прирост производительности не будет линейным.
import threading

def cpu_bound_task(n):
    total = 0
    for i in range(n):
        total += i * i
    return total

def run_in_thread(n):
    print(f"Result: {cpu_bound_task(n)}")

t1 = threading.Thread(target=run_in_thread, args=(10_000_000,))
t2 = threading.Thread(target=run_in_thread, args=(10_000_000,))

t1.start()
t2.start()
t1.join()
t2.join()
print("Done")

Result: 333333283333335000000
Result: 333333283333335000000
Done


## 2. Threads vs Processes

### 2.1. Потоки и процессы
- **Потоки (threads)**:
  - Делят общее адресное пространство.
  - Быстро создаются и переключаются.
  - Возникают сложности при синхронизации (race conditions).

- **Процессы (processes)**:
  - Имеют независимую память.
  - Создание тяжелее по ресурсам.
  - Легче изолировать, нет проблем с GIL (каждый процесс имеет свой интерпретатор).

### 2.2. RSS, VMS, Контекст
- **RSS (Resident Set Size)**: фактическое использование RAM.
- **VMS (Virtual Memory Size)**: общий объем виртуальной памяти, выделенной процессу.
- **Контекст**: набор данных (регистры, указатели), нужных для возобновления выполнения потока/процесса.

## 3. Модуль `threading`

### 3.1. `Lock`, `RLock`
- **`Lock`**: объект, который гарантирует эксклюзивный доступ к ресурсу.
- **`RLock`**: реентрантный лок, может быть захвачен одним и тем же потоком несколько раз.

In [2]:
import threading

lock = threading.Lock()
shared_counter = 0

def increment_counter():
    global shared_counter
    with lock:
        temp = shared_counter
        temp += 1
        shared_counter = temp

threads = [threading.Thread(target=increment_counter) for _ in range(1000)]
for t in threads:
    t.start()
for t in threads:
    t.join()

print("shared_counter =", shared_counter)

shared_counter = 1000


### 3.2. `Condition`
- Позволяет потокам **ждать** определенного условия и **уведомлять** друг друга. Имеет методы `wait()`, `notify()`, `notify_all()`.

### 3.3. `Semaphore`, `BoundedSemaphore`
- Ограничивают число потоков, которые могут использовать ресурс одновременно.
```python
sem = threading.Semaphore(3)  # Позволяем не более 3 потоков
```

### 3.4. `Event`
- Используется для уведомления других потоков о каком-то событии.
- `event.set()` – устанавливает событие.
- `event.wait()` – блокирует поток, пока событие не установлено.

### 3.5. `Barrier`
- Синхронизирует группу потоков, чтобы все «дошли до барьера» прежде, чем продолжить.

### 3.6. `Thread`
- Класс для создания потоков.
- `start()` — запускает поток.
- `join()` — ожидает завершение потока.

### 3.7. Race condition
- Возникает, когда результат зависит от **порядка** выполнения потоков.
- Может приводить к непредсказуемым ошибкам.
- Обычно решается с помощью `Lock`/`Semaphore`/`Condition` и т.д.

## 4. Модуль `multiprocessing`

- Позволяет создавать **процессы**, что обходит GIL.
- Аналогичен `threading`, но использует **процессы** под капотом.

### 4.1. `ProcessPoolExecutor`
Позволяет удобно параллелить задачи через пул процессов (внутри модуля `concurrent.futures`).

In [3]:
! python src/7/7_4_1.py

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801]


## 5. `asyncio`

Модуль для написания **асинхронного** кода в стиле кооперативной многозадачности.
- Подходит для IO-bound задач (сетевые запросы, ввод/вывод).

### 5.1. `await`, `asyncio.run()`
- `await` приостанавливает корутину, пока не будет готов **awaitable** объект.
- `asyncio.run(main())` запускает цикл событий и выполняет корутину `main` до конца.

In [4]:
! python src/7/7_5_1.py

Start task 1
Start task 2
Start task 3
End task 1
End task 2
End task 3


### 5.2. Event loop
- **Event loop** управляет выполнением корутин.
- Можно создавать (`new_event_loop()`) и устанавливать (`set_event_loop()`) свои циклы.

### 5.3. Корутины и Задачи
- **Корутины**: функции, определённые с `async def`, которые могут быть «приостановлены».
- **Задачи (Tasks)**: объекты, обёртывающие корутины. Создаются `asyncio.create_task(coroutine)`.

### 5.4. `asyncio.gather()`, колбэки
- `gather()` запускает несколько awaitable-объектов параллельно (кооперативно).
- Колбэки могут быть привязаны к задачам (не описаны подробно в источниках).

## Итог
Python предоставляет **разнообразные инструменты** для параллелизма и асинхронности, но **GIL** ограничивает параллельное исполнение байт-кода. Для **CPU-bound** задач эффективней использовать `multiprocessing` или внешние модули (NumPy/Cython и т.д.). Для **IO-bound** задач удобен `asyncio` или многопоточность. Выбор подхода зависит от конкретной задачи.